# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [16]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [17]:
#url
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#load csv from weatherpy
cities = "output_data/cities.csv"
cities_df = pd.read_csv(cities)
cities_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,jamestown,42.10,-79.24,45.00,70,90,9.17,US,1603681659
1,1,nikolskoye,59.70,30.79,43.00,93,90,8.95,RU,1603681664
2,2,padang,-0.95,100.35,83.70,60,100,2.53,ID,1603681594
3,3,cherskiy,68.75,161.30,16.18,96,100,2.57,RU,1603681627
4,4,vaini,-21.20,-175.20,73.40,94,90,10.29,TO,1603681681


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [18]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [19]:
#build heatmap
locations= cities_df[["Lat", "Lng"]].astype(float)
weight = cities_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weight, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
#ideal weather dataframe
ideal_weather=cities_df[cities_df["Max Temp"] >70][cities_df["Max Temp"]<90][cities_df["Wind Speed"]<15][cities_df["Cloudiness"]<20]
ideal_weather.head()

C:\Users\JMahon\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\JMahon\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\JMahon\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,50,sao filipe,20.51,76.21,75.47,67,0,3.06,IN,1603681819
55,55,lakki marwat,32.61,70.91,73.08,30,0,1.86,PK,1603681828
57,57,inhambane,-23.86,35.38,71.56,89,11,3.02,MZ,1603681833
62,62,bubaque,11.28,-15.83,83.12,79,0,7.61,GW,1603681841
74,74,hilo,19.73,-155.09,84.20,62,1,10.29,US,1603681730


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [21]:
#new hotel dataframe
hotel_df=ideal_weather.loc[:, ["City", "Country", "Lat", "Lng"]]
hotel_df ["Hotel Name"]= ""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
50,sao filipe,IN,20.51,76.21,
55,lakki marwat,PK,32.61,70.91,
57,inhambane,MZ,-23.86,35.38,
62,bubaque,GW,11.28,-15.83,
74,hilo,US,19.73,-155.09,


In [22]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [23]:
# Set parameters to search for a hotel
params = {"radius": 5000,
    "types": "lodging",
    "key": g_key}

# Iterate through 
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    
    response = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, 'Hotel'] = response['results'][0]['name']
    except (IndexError):
        print("Missing")

Missing
Missing
Missing
Missing
Missing


In [24]:
#look at new dataframe with names included
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name,Hotel
50,sao filipe,IN,20.51,76.21,,P.S.Shingne House
55,lakki marwat,PK,32.61,70.91,,Agriculture office lakki Marwat
57,inhambane,MZ,-23.86,35.38,,Oceano
62,bubaque,GW,11.28,-15.83,,"Casa Dora, Bubaque"
74,hilo,US,19.73,-155.09,,Hilo Hawaiian Hotel


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))